![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [396]:
# Importing necessary packages
import pandas as pd
import numpy as np
from thefuzz import process

In [397]:
# Importing CSV file
price_data = pd.read_csv("data/airbnb_price.csv")
price_data.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [398]:
# Importing Excel file
room_data = pd.read_excel("data/airbnb_room_type.xlsx")
room_data.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [399]:
# Importing TSV file
review_data = pd.read_csv("data/airbnb_last_review.tsv", sep="\t")
review_data.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [400]:
# Merging files
df = price_data.merge(room_data, how="inner", on="listing_id").merge(review_data, how="inner", on="listing_id")
df.head()

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt,Jennifer,May 21 2019
1,3831,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt,LisaRoxanne,July 05 2019
2,5099,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt,Chris,June 22 2019
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,June 24 2019
4,5238,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt,Ben,June 09 2019


In [401]:
df.dtypes

listing_id      int64
price          object
nbhood_full    object
description    object
room_type      object
host_name      object
last_review    object
dtype: object

In [402]:
#Converting 'last_review' column to datetime type
df["last_review"] = pd.to_datetime(df["last_review"], format="%B %d %Y")

In [403]:
# Earliest review
first_reviewed = df["last_review"].min()
print(first_reviewed)

2019-01-01 00:00:00


In [404]:
# Most recent review
last_reviewed = df["last_review"].max()
print(last_reviewed)

2019-07-09 00:00:00


In [405]:
# Categories
df["room_type"].value_counts()

Entire home/apt    8458
Private room       7241
entire home/apt    2665
private room       2248
ENTIRE HOME/APT    2143
PRIVATE ROOM       1867
Shared room         380
shared room         110
SHARED ROOM          97
Name: room_type, dtype: int64

In [406]:
# Fixing the categories
categories = ["Entire Home", "Private Room", "Shared Room"]
for rooms in categories:
    matches = process.extract(rooms, df["room_type"].unique(), limit= df.shape[0])
    for potential_match in matches:
        if potential_match[1] >= 80:
            df.loc[df["room_type"] == potential_match[0], "room_type"] = rooms
            
df["room_type"].value_counts()

Entire Home     13266
Private Room    11356
Shared Room       587
Name: room_type, dtype: int64

In [407]:
# Number of private rooms
room_types = df["room_type"].value_counts()
nb_private_rooms = room_types.iloc[1]
print(nb_private_rooms)

11356


In [408]:
# Converting the price to float
df["price"] = df["price"].str.replace("dollars", "")
df["price"] = df["price"].astype(float)
avg_price = df["price"].mean().round(2)
print(avg_price)

141.78


In [409]:
# Creating DataFrame from calculated values
data = {"first_reviewed": [first_reviewed],
       "last_reviewed": [last_reviewed],
       "nb_private_rooms": [nb_private_rooms],
       "avg_price": [avg_price]}

review_dates = pd.DataFrame(data)